## What is MDS

$\mbox{Side story : Can we rebuild the Euclidean coordinates when we only have the dissimilarity matrix of samples}$ 

$\mbox{Idea : MDS reproduce the spatially information in a lower dimension under the constraint of distance maintain}$

$\mbox{Let }$
$$
X_{n \times p} = 
\begin{bmatrix}
\cdots &  x_1 & \cdots \\
 & \vdots & & \\
\cdots & x_n & \cdots \\
\end{bmatrix}
\mbox{ and }
Z_{n \times p\prime} = 
\begin{bmatrix}
\cdots &  x_1 & \cdots \\
 & \vdots & & \\
\cdots & x_n & \cdots \\
\end{bmatrix}
$$

$\mbox{ be the matrix whose rows are samples and columns are features.}$
- $\mbox{In fact, we don't know the absolute position, i.e., }X,Z \mbox{ are unknown.}$
- $\mbox{We hope to remove some unimportant features in }X \mbox{ to obtain a dimension reduction matrix }Z$
- $p > p\prime$

$\mbox{Let }$
$$
D =
\begin{bmatrix}
0 & \cdots & d(x_1,x_n) \\
\vdots & \ddots & \vdots \\
d(x_n,x_1) & \cdots & 0 \\
\end{bmatrix}
$$

$\mbox{be the dissimilarity matirx, and }d_{ij} \mbox{ represents the distance between i-th sample and j-th sample}$

$\mbox{and let}$

$$
B = Z^\top Z = 
\begin{bmatrix}
z_1^\top z_1 & z_1^\top z_2 & \cdots & z_1^\top z_n \\
z_2^\top z_1 & z_2^\top z_2 & \cdots & z_2^\top z_n \\
\vdots & \vdots & \ddots & \vdots \\
z_n^\top z_1 & z_n^\top z_2 & \cdots & z_n^\top z_n
\end{bmatrix}
$$

$\mbox{be the inner production matrix}$

- $\mbox{Question : How to find matrix }B \mbox{ in term of }D \ ?$
- $\mbox{Let }H = I_n - \frac{1}{n}J_n \mbox{ be the projection matrix, but also the matrix normalize }D$ 
- $\mbox{Fact : Any symmetric matrix }M \mbox{ can be decomposition to }A^\top A \mbox{ for some matrix }A$
  - $M = PDP^\top = PD^{1/2}(PP^\top)D^{1/2}P^\top = A^\top (PP^\top)A = A^\top A \mbox{ where }A = D^{1/2}P^\top$
- $\mbox{Let }A \mbox{ be the matrix whose ij-entries are the square of }d_{ij}$
- $B = \frac{-1}{2}H^\top AH = \frac{-1}{2}HAH$

## Extend

$\mbox{Notice that if we have }n \mbox{ samples in }\mathbb{R}^{p} \mbox{ with equal distance, the maximum dimension which maintain distance after dimension reduction is }n-1$

$\bf\mbox{Stress : }$ $\bf\displaystyle Stress_D(z_1,\cdots,z_n)=\Biggl(\sum_{i\ne j=1,...,N}\bigl(d_{ij}-\|z_i-z_j\|\bigr)^2\Biggr)^{1/2}$

- $\mbox{the value of stress is the residual sum of squares}$
- $\mbox{the value of stress can be also considerd as the loss information}$

## Algorithm

$\bf\mbox{Input : }$

- `D` : $\mbox{An 2d array, which is a dissimilarity matrix}$ 
- `k` : $\mbox{An int object, which is the target dimension}$

$\bf\mbox{Output : }$

- `Z` : $\mbox{An 2d array, whose rows are samples and columns are features which have been reduced dimension}$

$\bf\mbox{Step : }$

1. $\mbox{Calculate }A \mbox{ from }$ `D` $\mbox{ by}$ `np.squared`
2. $\mbox{Calculate }H = I_n - \frac{1}{n}J_n$ $\mbox{by }$ `np.eye`, `np.ones`
3. $P = \frac{-1}{2}HAH$
4. $\mbox{Calculate eigenvalues and eigenvectors of }P \mbox{ by }$ `np.linalg.eigh`
5. $\mbox{Select exactly top k eigenvalues and corresponds eigenvectors}$
6. $\mbox{Let the top k eigenvalues in order be the diagonal of }\Lambda$
7. $\mbox{Let the eigenvectors corresponds to top k eigenvalues in order form a matrix }U$
8. $Z = \Lambda U$

## Code

```python
def dissimilarity(data, degree):
    m, n = data.shape[0], data.shape[1]
    x = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            x[i, j] = (np.sum(abs(data[i,:] - data[j,:]) ** degree)) ** (1 / degree)
            x[j, i] = x[i, j]
    return x
```

```python
def mds(D, k):
    n_samples = D.shape[0]
    DD = np.square(D)
    H = np.eye(n_samples) - np.ones((n_samples, n_samples)) / n_samples
    P = H.dot(DD).dot(H) / (-2)

    eigvals, eigvecs = np.linalg.eigh(P)
    eigvecs = eigvecs[:,::-1]
    eigvals = eigvals[::-1]
    Lambda = np.diag(eigvals[:k])
    Z = (Lambda ** 0.5).dot(eigvecs[:,:k].T)
    return Z
```

$\mbox{The code below is an alternative way to obtain inner product matrix }B \mbox{ in term of }A$

```python
def mds(D, k):
    n_samples = D.shape[0]
    dist_ij_square = np.square(D)
    dist_i_square = dist_ij_square.sum(axis = 1) / n_samples
    dist_j_square = dist_ij_square.sum(axis = 0) / n_samples
    dist_i_square = np.repeat(dist_i_square[:,np.newaxis], n_samples, axis = 1)
    dist_j_square = np.repeat(dist_j_square[np.newaxis,:], n_samples, axis = 0)
    dist_all_square = np.sum(dist_ij_square) / (n_samples ** 2)
    B = (dist_ij_square - dist_i_square - dist_j_square + dist_all_square) / (-2)
    eigvals, eigvecs = np.linalg.eigh(B)
    eigvecs = eigvecs[:,::-1]
    eigvals = eigvals[::-1]
    Lambda = np.diag(eigvals[:k])
    Z = (Lambda ** 0.5).dot(eigvecs[:,:k].T)
    return Z
```